# **Training Data Processing**

1. Project Setup
2. Prepare UCP Data
3. Rasterize UCP Data
4. Rasterize LCZ Data

# **1. Project Setup**

### 1.1. Import Libraries

In [1]:
%load_ext autoreload
%autoreload 

import sys
import os

# Add the module's parent directory to sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lcz_classification.config import * 
from lcz_classification.util import merge_rasters, resample_da, read_lcz_legend, rasterize_vector, generate_raster
from rioxarray.merge import merge_arrays
import rioxarray as rio
import pandas as pd
import rasterio as r
from shapely.geometry import box
import geopandas as gpd
import xarray as xr
from rasterio.enums import Resampling

### 1.2. Setup Paths

In [3]:
## WORFLOW PARAMETERS

## USER INPUT ##
# ===============================================================================================================================================================
# CELL_RESOLUTION = 30 # desired resolution in meters

S2_FILE_PATH=f"../data/processed/sentinel2/resampled/s2_20210714_{CELL_RESOLUTION}m.tif"


## PROJECT CONFIGURATION ##
# ===============================================================================================================================================================

# Establish Study Area bounds

STUDY_AREA_GDF=gpd.read_file(STUDY_AREA_FP)
STUDY_AREA_GDF=STUDY_AREA_GDF.to_crs(STUDY_AREA_GDF.estimate_utm_crs())

# Create Bounding Box Polygon
BBOX=box(*list(STUDY_AREA_GDF.total_bounds))
BBOX= BBOX.buffer(15, join_style=2) # Create 15 meter buffer for bounding box

# Retrieve CRS Based on bounds of study area
CRS=STUDY_AREA_GDF.estimate_utm_crs() # NAD83 / UTM zone 9N

LCZ_LEGEND, COLOR_DICT=read_lcz_legend(LCZ_LEGEND_FP)


### 1.3. Setup Reference Raster

In [ ]:
s2=rio.open_rasterio(S2_FILE_PATH)
s2_ref=s2.isel(band=0)

# **2. Urban Canopy Layer Processing**

## 2.1. Sky View Factor

#### 2.1.1. Merge  DSM Layers 

In [ ]:
dsm_files= [f"{DSM_DIR}/{tif}" for tif in os.listdir(DSM_DIR) if ".tif" in tif] # prepare file paths of DSM tiles

# Cke
for dsm_file in dsm_files:
    dsm=rio.open_rasterio(dsm_file)
    dsm=dsm.where(dsm> 0, drop=True).fillna(0)
    dsm.rio.to_raster(f'{PROCESSED_DIR}/dsm/{dsm_file.split("/")[-1]}')

dsm_files= [f"{f'{PROCESSED_DIR}/dsm'}/{tif}" for tif in os.listdir(f'{PROCESSED_DIR}/dsm') if ".tif" in tif] # prepare file paths of DSM tiles

out_path=f"{PROCESSED_DIR}/dsm/alos_dsm_{CELL_RESOLUTION}m.tif" 
if out_path in dsm_files:
    dsm_files.remove(out_path)

merge_rasters(raster_paths=dsm_files,
                        out_path=out_path
                        ) # merge into a single raster using merge_arrays()

#### 2.1.2. Compute Sky View Factor

Run ../lcz_classification/svf_qgis.py in QGIS v3.28 Python Console

#### 2.1.3. Reproject Match to S2 Data [DELETE?]

In [ ]:
# ref=rio.open_rasterio(BH_FP)
svf=rio.open_rasterio('../data/processed/svf/svf_30m_aligned.tif')
attrs=svf.attrs

svf = (svf - svf.min()) / (svf.max() - svf.min())
# svf.values[~mask] = 0.0
# svf.attrs=attrs

svf.rio.to_raster(SF_FP)

svf.close()

## 2.2. Impervious Surface Fraction

In [ ]:
isf_fp="../data/external/impervious_area/GISD30_1985_2020_b1.tif"

isf=rio.open_rasterio(isf_fp)

isf = isf.rio.reproject(dst_crs=CRS)
isf=resample_da(isf,CELL_RESOLUTION,Resampling.average)
isf=isf.rio.reproject_match(s2_ref)
isf.rio.to_raster(f"../data/processed/training_data/impervious_surface_fraction_{CELL_RESOLUTION}m.tif")
isf.close()

## 2.3. Tree Canopy Height

In [ ]:
from lcz_classification.util import resample_da

ch_files= [f"{CH_DIR}/{tif}" for tif in os.listdir(CH_DIR) if ".tif" in tif] # prepare file paths of DSM tiles

out_path="../data/processed/canopy_height/canopy_height.tif" 
ch_merged=merge_rasters(raster_paths=ch_files,
                        out_path=out_path
                        ) # merge into a single raster using merge_arrays()

ch_merged

In [ ]:
fp="../data/processed/canopy_height/canopy_height.tif"
out_path=f"../data/processed/training_data/canopy_height_{CELL_RESOLUTION}m.tif"


ch=rio.open_rasterio(fp)
ch=ch.where(ch < 255)
ch_resampled=ch.rio.reproject_match(s2_ref, resampling=Resampling.average)
ch_resampled.rio.to_raster(out_path)

ch.close()
ch_resampled.close()


## 2.4. Building Height

#### 2.2.1. Combine Vancouver Building Layers into Single GeoParquet File

In [ ]:
fp="../data/processed/building_height/building_heights.parquet"
out_path=f"../data/processed/building_height/building_height_5m.tif"

In [ ]:
BH_FILES=[f"{BH_DIR}/{gdb}/{gdb.replace('Autobuilding_BC_','').replace('_fgdb','.gdb')}" for gdb in os.listdir(BH_DIR)]
filter_out_yr='2016'
BBOX=box(*list(STUDY_AREA_GDF.to_crs(4617).total_bounds))
BBOX= BBOX.buffer(15, join_style=2) # Create 15 meter buffer for bounding box

bh_gdf_list=[gpd.read_file(file, bbox=BBOX) for file in BH_FILES]

bh_gdf_list=[gdf.to_crs(4617) for gdf in bh_gdf_list]
bh_gdf=pd.concat(bh_gdf_list).reset_index(drop=True)



bh_gdf=bh_gdf[bh_gdf.datemax.str.contains(filter_out_yr)]
bh_gdf.to_parquet(fp)


#### 2.2.2. Rasterize Building Height Data

In [ ]:

if os.path.exists(out_path):
    print("READING")
    bh=rio.open_rasterio(out_path)
else:
    bh_gdf=gpd.read_parquet(fp)
    print("Generating 5m raster")
    ref = generate_raster(STUDY_AREA_GDF.total_bounds, crs=CRS, resolution=5)
    # Rasterize building heights
    rasterize_vector(gdf=bh_gdf,
                    ref=ref,
                    attribute="heightmax",
                    crs=CRS,
                    out_path=out_path
                )
    
    bh=rio.open_rasterio(out_path)


bh_resampled=bh.rio.reproject_match(s2_ref, resampling=Resampling.average)
bh_resampled.rio.to_raster(f"../data/processed/training_data/building_height_{CELL_RESOLUTION}m.tif")

bh.close()
bh_resampled.close()

READING


## 2.5. Building Surface Fraction

In [8]:
fp=f"../data/processed/building_height/building_height_5m.tif"
bh=rio.open_rasterio(fp)

arr=bh.values
mask=~bh.where(bh > 0).isnull().values

arr[mask] = 1
arr[~mask] = 0

bh.values=arr
bsf=bh.rio.reproject_match(s2_ref, resampling=Resampling.average)
bsf.rio.to_raster(f"../data/processed/training_data/building_surface_fraction_{CELL_RESOLUTION}m.tif")

bh.close()
bsf.close()

## 3. Rasterize Local Climate Zone (LCZ) Training Areas

In [ ]:
lcz_gdf=gpd.read_file(LCZ_TRAINING_FP)

lcz_gdf=lcz_gdf.dropna(subset='geometry').reset_index(drop=True)


lcz_gdf["Class ID"] = lcz_gdf.Class.apply(lambda cl: LCZ_LEGEND.set_index("class").loc[cl]["class_id"])
lcz_gdf["Name"] = lcz_gdf.Class.apply(lambda cl: LCZ_LEGEND.set_index("class").loc[cl]["name"])

colors=lcz_gdf.Class.apply(lambda cl: LCZ_LEGEND.set_index("class").loc[cl].hex)

m = lcz_gdf.explore(color=COLOR_DICT, 
                column="Name", 
                legend=True, 
                tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                attr='Esri World Imagery'
                )

STUDY_AREA_GDF.explore(style={
                "fill": False,
                "color": "red"
                },
                m=m
)
m



In [11]:
lcz_version=LCZ_TRAINING_FP.split("/")[-1].split(".")[0]
rasterize_vector(gdf=lcz_gdf,
                 ref=s2_ref,
                 attribute="Class",
                 crs=CRS,
                 out_path=f"../data/processed/training_data/{lcz_version}_{CELL_RESOLUTION}m.tif",
                 )
